<a href="https://colab.research.google.com/github/dretcm/World-Cup-Challenge-2022-ACM/blob/main/worlds_cup_2k22_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and preproccesing data

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

results = pd.read_csv("InternationalFootballResults_1872to2022_results.csv")
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [102]:
def onehotencoder(countrys, word):
  output = []
  for country in countrys:
    if country == word:
      output.append(1)
    else:
      output.append(0)
  return output

countrys = list(set(list(results['home_team']) + list(results['away_team'])))
len(onehotencoder(countrys, 'Argentina'))

316

In [103]:
home = list(results['home_team'])
home_score = list(results['home_score'])

away = list(results['away_team'])
away_score = list(results['away_score'])

winners = [home[i] if home_score[i]>away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
losers = [home[i] if home_score[i]<away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
ties = [home[i] for i in range(len(home)) if home_score[i]==away_score[i]]
ties += [away[i] for i in range(len(away)) if home_score[i]==away_score[i]]

table_winners = {}
for c in countrys:
    table_winners[c] = winners.count(c)

table_losers = {}
for c in countrys:
    table_losers[c] = losers.count(c)

table_ties = {}
for c in countrys:
    table_ties[c] = ties.count(c)

table_plays = {}
for c in countrys:
  table_plays[c] = table_winners[c] + table_losers[c] + table_ties[c]


table_result = []   # 0(tie), 1(win team 1), 2(win team 2)
for i in range(len(home_score)):
  if home_score[i] == away_score[i]:
    table_result.append(0)
  elif home_score[i] > away_score[i]:
    table_result.append(1)
  else:
    table_result.append(2)


'''
for i in sorted(table, key= lambda x: table[x], reverse=True):
  print(i + ' win: ', table[i])
'''


def get_partys_teams(t1,t2):
  first = results[results['home_team']==t1]
  second = first[first['away_team']==t2]
  third = results[results['home_team']==t2]
  fourth = third[third['away_team']==t1]

  a = second[second['home_score']>second['away_score']].shape[0]
  a += fourth[fourth['home_score']<fourth['away_score']].shape[0]

  b = second[second['home_score']<second['away_score']].shape[0]
  b += fourth[fourth['home_score']>fourth['away_score']].shape[0]

  c = second[second['home_score']==second['away_score']].shape[0]
  c += fourth[fourth['home_score']==fourth['away_score']].shape[0]
  return [a,b,c]

get_partys_teams("Spain", "Brazil"), get_partys_teams("Brazil", "Spain")

([2, 5, 2], [5, 2, 2])

In [104]:
import json
# table_1_vs_2 = {c+"_"+c2:get_partys_teams(c,c2) for c in countrys for c2 in countrys if c!=c2}
# with open("data_1vs2.json", "w") as write_file:
#     json.dump(table_1_vs_2, write_file)

with open("data_1vs2.json", "r") as read_file:
    table_1_vs_2 = json.load(read_file)

In [98]:
table_1_vs_2['Spain_Brazil'], table_1_vs_2['Brazil_Spain']

([2, 5, 2], [5, 2, 2])

In [6]:
np.mean(list(table_plays.values())), np.std(list(table_plays.values()))

(278.8607594936709, 270.4569496698867)

In [7]:
max(table_plays.values()), len(home)

(1051, 44060)

In [105]:
def normalize(table, standar=False):
  mean = np.mean(list(table.values())) if standar else 0
  std =  np.std(list(table.values())) if standar else max(table.values())
  return mean, std

mean, std = normalize(table_plays)
played1 = [(table_plays[country]-mean)/std for country in home] 
played2 =  [(table_plays[country]-mean)/std for country in away]

mean, std = normalize(table_winners)
win1 =  [(table_winners[country]-mean)/std for country in home]
win2 = [(table_winners[country]-mean)/std for country in away]

mean, std = normalize(table_losers)
lose1 = [(table_losers[country]-mean)/std for country in home]
lose2 = [(table_losers[country]-mean)/std for country in away]


arr = []
for i in range(len(home)):
  arr.append(onehotencoder(countrys, home[i]) + onehotencoder(countrys, away[i]))
arr = np.array(arr)

titles = [country+' '+str(i) for i in range(1,3) for country in countrys]
onehot_countrys = pd.DataFrame(arr, columns=titles)

results['played 1'] = played1
results['played 2'] = played2
results['win 1'] = win1
results['win 2'] = win2
results['lose 1'] = lose1
results['lose 2'] = lose2

results['win_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][0], axis=1)
results['lose_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['away_team']+"_"+x['home_team']][0], axis=1)
results['tie_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][2], axis=1)
results['total_1_vs_2'] = results.apply(lambda x: x['win_1_vs_2']+ x['lose_1_vs_2'] + x['tie_1_vs_2'], axis=1)

max_win_vs = max(results['win_1_vs_2'])
results['win_1_vs_2'] = results['win_1_vs_2'].map(lambda x: x/max_win_vs)

max_lose_vs = max(results['lose_1_vs_2'])
results['lose_1_vs_2'] = results['lose_1_vs_2'].map(lambda x: x/max_lose_vs)

max_tie_vs = max(results['tie_1_vs_2'])
results['tie_1_vs_2'] = results['tie_1_vs_2'].map(lambda x: x/max_tie_vs)

max_total_vs = max(results['total_1_vs_2'])
results['total_1_vs_2'] = results['total_1_vs_2'].map(lambda x: x/max_total_vs)

results['result'] = table_result
results['difference'] = results.apply(lambda x: x['home_score'] - x['away_score'], axis=1)

results['neutral'] = results['neutral'].map(lambda x: 1 if x==True else 0)

results = pd.concat([results, onehot_countrys], axis=1)

#results['date'] = results['date'].map(lambda x: (int(str(x)[:4])-1872)/2022)
#results = results.drop(['city', 'country', 'tournament', 'home_score', 'away_score', 'home_team', 'away_team'], axis=1)

# model 1:
data_1 = results.copy()
data_1.where(data_1['home_score']!=data_1['away_score'] ,inplace=True)
data_1 = data_1.drop(['city', 'country', 'date', 'tournament', 'home_score','away_score', 'home_team', 
                      'away_team','tie_1_vs_2','difference'], axis=1)
data_1 = data_1.dropna(axis=0)
data_1

,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,win_1_vs_2,lose_1_vs_2,total_1_vs_2,...,Guam 2,Two Sicilies 2,United Koreans in Japan 2,Eswatini 2,Ivory Coast 2,Syria 2,Saarland 2,Aymara 2,Chagos Islands 2,Guyana 2
1,0.0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.571429,0.500000,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.500000,0.571429,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.500000,0.571429,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.773549,0.655566,0.592934,0.339478,0.606280,0.765700,0.726190,0.273810,0.592179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.571429,0.500000,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44052,0.0,0.565176,0.224548,0.348694,0.090630,0.471014,0.309179,0.035714,0.011905,0.022346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44055,0.0,0.791627,0.290200,0.471582,0.213518,0.811594,0.210145,0.023810,0.035714,0.033520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44057,0.0,0.065652,0.371075,0.039939,0.112135,0.072464,0.606280,0.023810,0.000000,0.011173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44058,0.0,0.563273,0.640343,0.341014,0.264209,0.536232,0.835749,0.071429,0.035714,0.050279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,played 1,...,Guam 2,Two Sicilies 2,United Koreans in Japan 2,Eswatini 2,Ivory Coast 2,Syria 2,Saarland 2,Aymara 2,Chagos Islands 2,Guyana 2
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,0,0.773549,...,0,0,0,0,0,0,0,0,0,0
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,0,0.993340,...,0,0,0,0,0,0,0,0,0,0
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,0,0.773549,...,0,0,0,0,0,0,0,0,0,0
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,0,0.993340,...,0,0,0,0,0,0,0,0,0,0
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,0,0.773549,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,0,0.791627,...,0,0,0,0,0,0,0,0,0,0
44056,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,0,1.000000,...,0,0,0,0,0,0,0,0,0,0
44057,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,0,0.065652,...,0,0,0,0,0,0,0,0,0,0
44058,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,0,0.563273,...,0,0,0,0,0,0,0,0,0,0


In [107]:
# Data for model 1:
Y = data_1['result']
X = data_1.drop(['result'], axis=1)
X

,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,win_1_vs_2,lose_1_vs_2,total_1_vs_2,...,Guam 2,Two Sicilies 2,United Koreans in Japan 2,Eswatini 2,Ivory Coast 2,Syria 2,Saarland 2,Aymara 2,Chagos Islands 2,Guyana 2
1,0.0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.571429,0.500000,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.500000,0.571429,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.500000,0.571429,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.773549,0.655566,0.592934,0.339478,0.606280,0.765700,0.726190,0.273810,0.592179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.571429,0.500000,0.653631,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44052,0.0,0.565176,0.224548,0.348694,0.090630,0.471014,0.309179,0.035714,0.011905,0.022346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44055,0.0,0.791627,0.290200,0.471582,0.213518,0.811594,0.210145,0.023810,0.035714,0.033520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44057,0.0,0.065652,0.371075,0.039939,0.112135,0.072464,0.606280,0.023810,0.000000,0.011173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44058,0.0,0.563273,0.640343,0.341014,0.264209,0.536232,0.835749,0.071429,0.035714,0.050279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training NN-winner model

In [108]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
import keras

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(Y)
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [109]:
num_classes = y_train.shape[1]
num_input = X.shape[1]

model_winner = Sequential()
model_winner.add(Dense(1024, input_dim=num_input, activation='relu'))
model_winner.add(Dense(2048, activation='relu'))
model_winner.add(Dense(1024, activation='relu'))
model_winner.add(Dense(512, activation='relu'))
model_winner.add(Dense(256, activation='relu'))
model_winner.add(Dense(num_classes, activation='sigmoid'))

#model_winner.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_winner.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
model_winner.fit(X, y_train, epochs=150, batch_size=64)
model_winner.save('my_model_winner.h5')

Epoch 1/150
530/530 [==============================] - 3s 4ms/step - loss: 0.5282 - accuracy: 0.7356
Epoch 2/150
530/530 [==============================] - 2s 4ms/step - loss: 0.4805 - accuracy: 0.7677
Epoch 3/150
530/530 [==============================] - 2s 4ms/step - loss: 0.4588 - accuracy: 0.7785
Epoch 4/150
530/530 [==============================] - 2s 4ms/step - loss: 0.4388 - accuracy: 0.7876
Epoch 5/150
530/530 [==============================] - 2s 4ms/step - loss: 0.4181 - accuracy: 0.7943
Epoch 6/150
530/530 [==============================] - 2s 4ms/step - loss: 0.3999 - accuracy: 0.8008
Epoch 7/150
530/530 [==============================] - 2s 4ms/step - loss: 0.3828 - accuracy: 0.8063
Epoch 8/150
530/530 [==============================] - 2s 4ms/step - loss: 0.3694 - accuracy: 0.8109
Epoch 9/150
530/530 [==============================] - 2s 4ms/step - loss: 0.3589 - accuracy: 0.8162
Epoch 10/150
530/530 [==============================] - 2s 4ms/step - loss: 0.3495 - accura

In [110]:
def convert_input_winner(country_1, country_2, paises=countrys, house='Qatar'):
  #out = [1.0] # date is 2022
  out = []
  out.append(0 if country_1==house else 1)  # neutral: True/False : si jugarán en un lugar neutro, como es el mundial, todos juegan así.

  mean, std = normalize(table_plays)
  out.append((table_plays[country_1]-mean)/std)
  out.append((table_plays[country_2]-mean)/std)

  mean, std = normalize(table_winners)
  out.append((table_winners[country_1]-mean)/std)
  out.append((table_winners[country_2]-mean)/std)
  
  mean, std = normalize(table_losers)
  out.append((table_losers[country_1]-mean)/std)
  out.append((table_losers[country_2]-mean)/std)

  reply = table_1_vs_2[country_1+"_"+country_2]
  out.append(reply[0]/max_win_vs)
  out.append(reply[1]/max_lose_vs)
  out.append(sum(reply)/max_total_vs)

  out += onehotencoder(paises,country_1) + onehotencoder(paises, country_2)

  return np.array([out])

test = convert_input_winner("Bolivia", "Brazil")
test

array([[1.        , 0.4643197 , 0.96669838, 0.16743472, 1.        ,
        0.63768116, 0.38647343, 0.05952381, 0.27380952, 0.17877095,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [111]:
model_winner = load_model('my_model_winner.h5')
test = convert_input_winner("Brazil", "Peru")
print(model_winner.predict(test))
test = convert_input_winner("Peru", "Brazil")
model_winner.predict(test)

1/1 [==============================] - 0s 61ms/step
[[0.16186357]]
1/1 [==============================] - 0s 14ms/step


array([[1.]], dtype=float32)

In [112]:
def predict_winner(country_1, country_2, model, tie = True, ver=False):
  party_1 = model.predict(convert_input_winner(country_1, country_2), verbose=0)[0][0]
  party_2 = model.predict(convert_input_winner(country_2, country_1), verbose=0)[0][0]
  if ver:
    print(party_1, party_2)
  if (party_1<0.51) == (party_2<0.51):
    if tie:
      return 'Tie'
    else:
      if party_1>party_2:
        return country_2
      return country_1
  elif (party_1<0.51) and (party_2>0.51):
    return country_1
  return country_2


  # party_1[1] += party_2[2]
  # party_1[2] += party_2[1]
  # if tie:
  #   pos = np.argmax(party_1)
  # else:
  #   pos = np.argmax(party_1[1:]) + 1
  # if pos == 0:
  #   return "tie"
  #return country_1 if pos==1 else country_2
  

predict_winner("Brazil", "Peru", model_winner), predict_winner("Argentina", "Peru", model_winner, False, True), predict_winner("Peru","Argentina", model_winner, False, True)

0.1347257 3.4263165e-05
3.4263165e-05 0.1347257


('Brazil', 'Peru', 'Peru')

# Training NN-gols model

In [113]:
# data for model 2:
results = pd.read_csv("InternationalFootballResults_1872to2022_results.csv")

home = list(results['home_team'])
home_score = list(results['home_score'])

away = list(results['away_team'])
away_score = list(results['away_score'])

home,away = home+away, away+home
home_score,away_score = home_score+away_score, away_score+home_score

winners = [home[i] if home_score[i]>away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
losers = [home[i] if home_score[i]<away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
ties = [home[i] for i in range(len(home)) if home_score[i]==away_score[i]]
ties += [away[i] for i in range(len(away)) if home_score[i]==away_score[i]]

table_winners = {}
for c in countrys:
    table_winners[c] = winners.count(c)//2

table_losers = {}
for c in countrys:
    table_losers[c] = losers.count(c)//2

table_ties = {}
for c in countrys:
    table_ties[c] = ties.count(c)//2

table_plays = {}
for c in countrys:
  table_plays[c] = table_winners[c] + table_losers[c] + table_ties[c]


table_result = []   # 0(tie), 1(win team 1), 2(win team 2)
for i in range(len(home_score)):
  if home_score[i] == away_score[i]:
    table_result.append(0)
  elif home_score[i] > away_score[i]:
    table_result.append(1)
  else:
    table_result.append(2)

mean, std = normalize(table_plays)
played1 = [(table_plays[country]-mean)/std for country in home]
played2 =  [(table_plays[country]-mean)/std for country in away]

mean, std = normalize(table_winners)
win1 =  [(table_winners[country]-mean)/std for country in home]
win2 = [(table_winners[country]-mean)/std for country in away]

mean, std = normalize(table_losers)
lose1 = [(table_losers[country]-mean)/std for country in home]
lose2 = [(table_losers[country]-mean)/std for country in away]

mean, std = normalize(table_ties)
tie1 = [(table_ties[country]-mean)/std for country in home]
tie2 = [(table_ties[country]-mean)/std for country in away]

arr = []
for i in range(len(home)):
  arr.append(onehotencoder(countrys, home[i]) + onehotencoder(countrys, away[i]))
arr = np.array(arr)

titles = [country+' '+str(i) for i in range(1,3) for country in countrys]
onehot_countrys = pd.DataFrame(arr, columns=titles)

results_2 = pd.concat([results, results], axis=0)

results_2['played 1'] = played1
results_2['played 2'] = played2
results_2['win 1'] = win1
results_2['win 2'] = win2
results_2['lose 1'] = lose1
results_2['lose 2'] = lose2
results_2['tie 1'] = tie1
results_2['tie 2'] = tie2


results_2['win_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][0], axis=1)
results_2['lose_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['away_team']+"_"+x['home_team']][0], axis=1)
results_2['tie_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][2], axis=1)
results_2['total_1_vs_2'] = results_2.apply(lambda x: x['win_1_vs_2']+ x['lose_1_vs_2'] + x['tie_1_vs_2'], axis=1)

max_win_vs = max(results_2['win_1_vs_2'])
results_2['win_1_vs_2'] = results_2['win_1_vs_2'].map(lambda x: x/max_win_vs)

max_lose_vs = max(results_2['lose_1_vs_2'])
results_2['lose_1_vs_2'] = results_2['lose_1_vs_2'].map(lambda x: x/max_lose_vs)

max_tie_vs = max(results_2['tie_1_vs_2'])
results_2['tie_1_vs_2'] = results_2['tie_1_vs_2'].map(lambda x: x/max_tie_vs)

max_total_vs = max(results_2['total_1_vs_2'])
results_2['total_1_vs_2'] = results_2['total_1_vs_2'].map(lambda x: x/max_total_vs)

results_2['result'] = table_result
#results_2['difference'] = results_2.apply(lambda x: x['home_score'] - x['away_score'], axis=1)
results_2['gols team 1'] = home_score

results_2['neutral'] = results_2['neutral'].map(lambda x: 1 if x==True else 0)

#print(results_2.shape, onehot_countrys.shape)
results_2.reset_index(inplace=True, drop=True)
onehot_countrys.reset_index(inplace=True, drop=True)
results_2 = pd.concat([results_2, onehot_countrys], axis=1)

data_2 = results_2.drop(['city', 'country', 'date', 'tournament','home_score', 'away_score', 'home_team', 'away_team'], axis=1)

data_2

,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,tie 1,tie 2,win_1_vs_2,...,Guam 2,Two Sicilies 2,United Koreans in Japan 2,Eswatini 2,Ivory Coast 2,Syria 2,Saarland 2,Aymara 2,Chagos Islands 2,Guyana 2
0,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,0,0,0
1,0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.996016,0.701195,0.571429,...,0,0,0,0,0,0,0,0,0,0
2,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,0,0,0
3,0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.996016,0.701195,0.571429,...,0,0,0,0,0,0,0,0,0,0
4,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88115,0,0.290200,0.791627,0.213518,0.471582,0.210145,0.811594,0.314741,0.752988,0.023810,...,0,0,0,0,0,0,0,0,0,0
88116,0,0.261656,1.000000,0.147465,0.792627,0.260870,0.743961,0.282869,0.904382,0.023810,...,0,0,0,0,0,0,0,0,0,0
88117,0,0.371075,0.065652,0.112135,0.039939,0.606280,0.072464,0.262948,0.051793,0.023810,...,0,0,0,0,0,0,0,0,0,0
88118,0,0.640343,0.563273,0.264209,0.341014,0.835749,0.536232,0.617530,0.589641,0.071429,...,0,0,0,0,0,0,0,0,0,0


In [114]:
Y = data_2['gols team 1']
X = data_2.drop(['gols team 1','result'], axis=1)
X.insert(X.shape[1], "party tie", data_2['result'].map(lambda x: 0 if x!=0 else 1))
X.insert(X.shape[1], "win country 1", data_2['result'].map(lambda x: 0 if x!=1 else 1))
X.insert(X.shape[1], "win country 2", data_2['result'].map(lambda x: 0 if x!=2 else 1))
X.head()

,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,tie 1,tie 2,win_1_vs_2,...,Eswatini 2,Ivory Coast 2,Syria 2,Saarland 2,Aymara 2,Chagos Islands 2,Guyana 2,party tie,win country 1,win country 2
0,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,1,0,0
1,0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.996016,0.701195,0.571429,...,0,0,0,0,0,0,0,0,1,0
2,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,0,1,0
3,0,0.993340,0.773549,0.912442,0.592934,0.483092,0.606280,0.996016,0.701195,0.571429,...,0,0,0,0,0,0,0,1,0,0
4,0,0.773549,0.993340,0.592934,0.912442,0.606280,0.483092,0.701195,0.996016,0.500000,...,0,0,0,0,0,0,0,0,1,0


In [115]:
Y = Y.map(lambda x: abs(x))
Y

0        0.0
1        4.0
2        2.0
3        2.0
4        3.0
        ... 
88115    2.0
88116    1.0
88117    1.0
88118    1.0
88119    NaN
Name: gols team 1, Length: 88120, dtype: float64

In [116]:
Y = Y.fillna(0)
Y

0        0.0
1        4.0
2        2.0
3        2.0
4        3.0
        ... 
88115    2.0
88116    1.0
88117    1.0
88118    1.0
88119    0.0
Name: gols team 1, Length: 88120, dtype: float64

In [117]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
import keras

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(Y)
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [118]:
y_train.shape

(88120, 26)

In [120]:
num_classes = y_train.shape[1]
num_input = X.shape[1]

model = Sequential()
model.add(Dense(1024, input_dim=num_input, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y_train, epochs=200, batch_size=64)

Epoch 1/200
1377/1377 [==============================] - 8s 5ms/step - loss: 1.3002 - accuracy: 0.4411
Epoch 2/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.2635 - accuracy: 0.4510
Epoch 3/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.2477 - accuracy: 0.4575
Epoch 4/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.2305 - accuracy: 0.4627
Epoch 5/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.2073 - accuracy: 0.4729
Epoch 6/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.1779 - accuracy: 0.4860
Epoch 7/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.1451 - accuracy: 0.5012
Epoch 8/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.1122 - accuracy: 0.5173
Epoch 9/200
1377/1377 [==============================] - 6s 5ms/step - loss: 1.0761 - accuracy: 0.5315
Epoch 10/200
1377/1377 [==============================] - 6s 5ms/step - l

In [121]:
model.save('my_model_gols.h5')
clases = label_binarizer.classes_
clases

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 24., 30., 31.])

In [122]:
def convert_input_winner_2(country_1, country_2, paises=countrys, house='Qatar'):
  #out = [1.0] # date is 2022
  out = []
  out.append(0 if country_1==house or country_2==house else 1)  # neutral: True/False : si jugarán en un lugar neutro, como es el mundial, todos juegan así.

  mean, std = normalize(table_plays)
  out.append((table_plays[country_1]-mean)/std)
  out.append((table_plays[country_2]-mean)/std)

  mean, std = normalize(table_winners)
  out.append((table_winners[country_1]-mean)/std)
  out.append((table_winners[country_2]-mean)/std)
  
  mean, std = normalize(table_losers)
  out.append((table_losers[country_1]-mean)/std)
  out.append((table_losers[country_2]-mean)/std)

  mean, std = normalize(table_ties)
  out.append((table_ties[country_1]-mean)/std)
  out.append((table_ties[country_2]-mean)/std)

  reply = table_1_vs_2[country_1+"_"+country_2]
  out.append(reply[0]/max_win_vs)
  out.append(reply[1]/max_lose_vs)

  out.append(reply[2]/max_tie_vs)
  out.append(sum(reply)/max_total_vs)

  out += onehotencoder(paises,country_1) + onehotencoder(paises, country_2)

  return np.array([out])

def predict_results_full(country_1, country_2, model_winner, model, clases, tie=True):
  test = list(convert_input_winner_2(country_1, country_2)[0])
  test_2 = list(convert_input_winner_2(country_2, country_1)[0])
  win = predict_winner(country_1, country_2, model_winner, tie)
  if win == 'Tie':
    win = country_1 + "_" + country_2
    test.extend([1,0,0])
    test_2.extend([1,0,0])
  elif win==country_1:
    win += "_"+country_2
    test.extend([0,1,0])
    test_2.extend([0,0,1])
  else:
    win += "_"+country_1
    test.extend([0,0,1])
    test_2.extend([0,1,0])

  test = np.array([test])
  test_2 = np.array([test_2])

  pred_1 = model.predict(test, verbose=0)[0]
  pred_2 = model.predict(test_2, verbose=0)[0]
  pos_1 = np.argmax(pred_1)
  pos_2 = np.argmax(pred_2)
  if not tie and pos_1==pos_2:
    if pred_1[pos_1]>pred_2[pos_2]:
      b = pred_1[pos_1]
      a = pred_1
      mx = -10000000
      for i in a:
        if i>mx and i!= b:
          mx = i
      for i in range(len(a)):
        if a[i]==mx:
          pos_1 = i
          break
    else:
      b = pred_2[pos_2]
      a = pred_2
      mx = -10000000
      for i in a:
        if i>mx and i!= b:
          mx = i
      for i in range(len(a)):
        if a[i]==mx:
          pos_2 = i
          break
  prediction = clases[pos_1]
  prediction_2 = clases[pos_2]
  return country_1 + " ("+str(int(prediction))+"-"+str(int(prediction_2))+") " + country_2

predict_results_full("Brazil", "Peru", model_winner, model, clases)

'Brazil (2-0) Peru'

In [123]:
predict_results_full("Peru", "Brazil", model_winner, model, clases), predict_results_full("Brazil", "Peru", model_winner, model, clases)

('Peru (0-2) Brazil', 'Brazil (2-0) Peru')

In [124]:
predict_results_full("Chile", "Peru", model_winner, model, clases), predict_results_full("Peru", "Chile", model_winner, model, clases)

('Chile (0-1) Peru', 'Peru (1-0) Chile')

In [125]:
predict_results_full("Qatar", "Peru", model_winner, model, clases), predict_results_full("Peru", "Qatar", model_winner, model, clases)

('Qatar (0-2) Peru', 'Peru (2-0) Qatar')

In [126]:
predict_results_full("Argentina", "Peru", model_winner, model, clases), predict_results_full("Peru","Argentina", model_winner, model, clases)

('Argentina (2-0) Peru', 'Peru (0-2) Argentina')

In [127]:
predict_results_full("Brazil", "Bolivia", model_winner, model, clases), predict_results_full("Bolivia", "Brazil", model_winner, model, clases)

('Brazil (8-0) Bolivia', 'Bolivia (0-8) Brazil')

In [128]:
def predict_results(country_1, country_2, model_winner, model, clases, tie=True):
  cad = predict_results_full(country_1, country_2, model_winner, model, clases, tie).split("-")
  team_1 = int(cad[0][-1])
  team_2 = int(cad[1][0])

  if team_1 > team_2:
    return cad[0][:len(cad[0])-3]+"_"+cad[1][3:]+"_+"+str(team_1-team_2)
  else:
    return cad[1][3:]+"_"+cad[0][:len(cad[0])-3]+"_+"+str(team_2-team_1)

predict_results("Brazil", "Bolivia", model_winner, model, clases)

'Brazil_Bolivia_+8'

In [129]:
predict_results("Chile", "Peru", model_winner, model, clases, False)

'Peru_Chile_+3'

# Grupos

In [130]:
Groups = {"Group A": ["Qatar",  "Ecuador", "Senegal" ,"Netherlands"],
          "Group B": ["England",  "Iran", "United States",  "Wales"],
          "Group C": ["Argentina",  "Saudi Arabia",  "Mexico",  "Poland"],
          "Group D": ["France",  "Australia",  "Denmark",  "Tunisia"],
          "Group E": ["Spain",  "Costa Rica",  "Germany",  "Japan"],
          "Group F": ["Belgium",  "Canada",  "Morocco", "Croatia"],
          "Group G": ["Brazil", "Serbia",  "Switzerland",  "Cameroon"],
          "Group H": ["Portugal",  "Ghana", "Uruguay",  "South Korea"]}
Groups

{'Group A': ['Qatar', 'Ecuador', 'Senegal', 'Netherlands'],
 'Group B': ['England', 'Iran', 'United States', 'Wales'],
 'Group C': ['Argentina', 'Saudi Arabia', 'Mexico', 'Poland'],
 'Group D': ['France', 'Australia', 'Denmark', 'Tunisia'],
 'Group E': ['Spain', 'Costa Rica', 'Germany', 'Japan'],
 'Group F': ['Belgium', 'Canada', 'Morocco', 'Croatia'],
 'Group G': ['Brazil', 'Serbia', 'Switzerland', 'Cameroon'],
 'Group H': ['Portugal', 'Ghana', 'Uruguay', 'South Korea']}

In [131]:
import itertools
list(itertools.combinations(Groups["Group A"], 2))

[('Qatar', 'Ecuador'),
 ('Qatar', 'Senegal'),
 ('Qatar', 'Netherlands'),
 ('Ecuador', 'Senegal'),
 ('Ecuador', 'Netherlands'),
 ('Senegal', 'Netherlands')]

In [132]:
def win_groups(group_X):
  print(" =====================>",group_X)
  partys = list(itertools.combinations(Groups[group_X], 2))
  match = []
  for party in partys:
    print(predict_results_full(party[0], party[1], model_winner, model, clases))
    match.append(predict_results(party[0], party[1], model_winner, model, clases))
  
  positions = dict(zip(Groups[group_X], [{"win":0.0,"gols":0.0}, {"win":0.0,"gols":0.0}, {"win":0.0,"gols":0.0}, {"win":0.0,"gols":0.0}]))
  print()
  for i in match:

    print(i)
    replys = i.split("_")

    if float(replys[2][1:])!=0.0:
      positions[replys[0]]["win"] += 1
      positions[replys[0]]["gols"] += float(replys[2][1:])

  print()
  for i in positions:
    print(i, positions[i])
  return sorted(positions, key=lambda x: positions[x]["win"]+(positions[x]["gols"]/100), reverse=True)[:2]

In [133]:
octavos = {}
for group in Groups:
  reply = win_groups(group)
  print(reply,"\n","-"*50)
  octavos[group] = reply

 =====================> Group A
Qatar (1-1) Ecuador
Qatar (2-1) Senegal
Qatar (0-2) Netherlands
Ecuador (0-2) Senegal
Ecuador (1-1) Netherlands
Senegal (2-0) Netherlands

Ecuador_Qatar_+0
Qatar_Senegal_+1
Netherlands_Qatar_+2
Senegal_Ecuador_+2
Netherlands_Ecuador_+0
Senegal_Netherlands_+2

Qatar {'win': 1.0, 'gols': 1.0}
Ecuador {'win': 0.0, 'gols': 0.0}
Senegal {'win': 2.0, 'gols': 4.0}
Netherlands {'win': 1.0, 'gols': 2.0}
['Senegal', 'Netherlands'] 
 --------------------------------------------------
 =====================> Group B
England (1-0) Iran
England (1-1) United States
England (2-0) Wales
Iran (1-1) United States
Iran (1-3) Wales
United States (2-0) Wales

England_Iran_+1
United States_England_+0
England_Wales_+2
United States_Iran_+0
Wales_Iran_+2
United States_Wales_+2

England {'win': 2.0, 'gols': 3.0}
Iran {'win': 0.0, 'gols': 0.0}
United States {'win': 1.0, 'gols': 2.0}
Wales {'win': 1.0, 'gols': 2.0}
['England', 'United States'] 
 ------------------------------------

#Octavos

In [134]:
octavos

{'Group A': ['Senegal', 'Netherlands'],
 'Group B': ['England', 'United States'],
 'Group C': ['Argentina', 'Poland'],
 'Group D': ['Tunisia', 'France'],
 'Group E': ['Spain', 'Germany'],
 'Group F': ['Belgium', 'Croatia'],
 'Group G': ['Brazil', 'Cameroon'],
 'Group H': ['Uruguay', 'Portugal']}

In [135]:
oct = list(octavos.keys())
cuartos_replys = []

for i in range(0, len(oct),2):
  cuartos_replys.append(predict_results(octavos[oct[i]][0], octavos[oct[i+1]][1], model_winner, model, clases, tie=False))
  cuartos_replys.append(predict_results(octavos[oct[i]][1], octavos[oct[i+1]][0], model_winner, model, clases, tie=False))
  print(predict_results_full(octavos[oct[i]][0], octavos[oct[i+1]][1], model_winner, model, clases, tie=False), end=" | ")
  print(cuartos_replys[i])
  print(predict_results_full(octavos[oct[i]][1], octavos[oct[i+1]][0], model_winner, model, clases, tie=False), end= " | ")
  print(cuartos_replys[i+1])

Senegal (2-1) United States | Senegal_United States_+1
Netherlands (3-1) England | Netherlands_England_+2
Argentina (1-0) France | Argentina_France_+1
Poland (1-0) Tunisia | Poland_Tunisia_+1
Spain (2-0) Croatia | Spain_Croatia_+2
Germany (3-2) Belgium | Germany_Belgium_+1
Brazil (3-0) Portugal | Brazil_Portugal_+3
Cameroon (0-4) Uruguay | Uruguay_Cameroon_+4


# Cuartos

In [136]:
cuartos = [line.split("_")[0] for line in cuartos_replys]
cuartos

['Senegal',
 'Netherlands',
 'Argentina',
 'Poland',
 'Spain',
 'Germany',
 'Brazil',
 'Uruguay']

In [137]:
segundos_replys = []

for i in range(0, len(cuartos),2):
  print(predict_results_full(cuartos[i], cuartos[i+1], model_winner, model, clases, tie=False), end = " | ")
  segundos_replys.append(predict_results(cuartos[i], cuartos[i+1], model_winner, model, clases, tie=False))
  print(segundos_replys[-1])

Senegal (1-4) Netherlands | Netherlands_Senegal_+3
Argentina (2-1) Poland | Argentina_Poland_+1
Spain (1-0) Germany | Spain_Germany_+1
Brazil (3-0) Uruguay | Brazil_Uruguay_+3


# Semi-final & Final

In [138]:
segundos = [line.split("_")[0] for line in segundos_replys]
segundos

['Netherlands', 'Argentina', 'Spain', 'Brazil']

In [139]:
semiF_replys = []

for i in range(0, len(segundos),2):
  semiF_replys.append(predict_results(segundos[i], segundos[i+1], model_winner, model, clases, tie=False))
  print(predict_results_full(segundos[i], segundos[i+1], model_winner, model, clases, tie=False), end=" | ")
  print(semiF_replys[-1])

pos4_and_3 = [line.split("_")[1] for line in semiF_replys]
tercero = predict_results(pos4_and_3[0], pos4_and_3[1], model_winner, model, clases, tie=False).split("_")[0]
pos4_and_3.remove(tercero)

print("\nFourth in the world cup:", pos4_and_3[0])
print("Third in the world cup:", tercero)

semiF = [line.split("_")[0] for line in semiF_replys]
print("\nThe Final is between:",semiF)

Netherlands (4-0) Argentina | Netherlands_Argentina_+4
Spain (0-2) Brazil | Brazil_Spain_+2

Fourth in the world cup: Spain
Third in the world cup: Argentina

The Final is between: ['Netherlands', 'Brazil']


In [140]:
# Final:

final = predict_results(semiF[0], semiF[1], model_winner, model, clases, tie=False)
print(predict_results_full(semiF[0], semiF[1], model_winner, model, clases, tie=False), end=" | ")
print(final)

semiF.remove(final.split("_")[0])
print("\nSecond in the world cup:", semiF[0])
print("\nWorld Cup Winner is: ", final.split("_")[0])

Netherlands (2-0) Brazil | Netherlands_Brazil_+2

Second in the world cup: Brazil

World Cup Winner is:  Netherlands
